## Challenge

Given columnar data format vicol1 compare its reading performance with CSV table. 
Check select and where clause.

In [4]:
from vicol1_table import ViCol1TableReader, ViCol1TableWriter
from csv_table import CsvTable

from faker import Faker
faker = Faker('en')

import numpy as np


# Prepare data

Prepare 1000000 records with 20 columns and write them to employees.vicol & employees.csv.

In [5]:
def generate_employee(salary):
    return {
        'first_name': faker.first_name(), 
        'last_name': faker.last_name(), 
        'department': faker.random_element(elements=['Embedded','BigData','HR']), 
        'salary': salary,
        'address': faker.street_address(),
        'city': faker.city(),
        'country': faker.country(),
        'address2': faker.street_address(),
        'city2': faker.city(),
        'country2': faker.country(),
        'notes': faker.catch_phrase(),
        'age': faker.random_int(max=60,min=18),
        'birth_location': faker.city(),
        'points': faker.random_int(min=0, max=300),
        'gender': faker.random_element(elements=['Female', 'Male']),
        'mothers_name': faker.first_name_female(),
        'fathers_name': faker.last_name_male(),
        'ID': faker.uuid4(),
        'pets_name': faker.first_name(),
        'drivers_license': faker.random_element(elements=['B', 'C', 'C+E', 'None'])
    }

In [36]:
# generate employees

salaries = np.random.normal(15000, 4000, 1000000)
employees = []
for salary in salaries:
    employees.append(generate_employee(salary))

# write vicol1
        
with open('resources/employees.vicol1', 'w') as f:
    f.write('{"columns":{"first_name":{"start":0,"end":0},"last_name":{"start":0,"end":0},"department":{"start":0,"end":0},"salary":{"start":0,"end":0},"address":{"start":0,"end":0},"city":{"start":0,"end":0},"country":{"start":0,"end":0},"address2":{"start":0,"end":0},"city2":{"start":0,"end":0},"country2":{"start":0,"end":0},"notes":{"start":0,"end":0},"age":{"start":0,"end":0},"birth_location":{"start":0,"end":0},"points":{"start":0,"end":0},"gender":{"start":0,"end":0},"mothers_name":{"start":0,"end":0},"fathers_name":{"start":0,"end":0},"ID":{"start":0,"end":0},"pets_name":{"start":0,"end":0},"drivers_license":{"start":0,"end":0}}}\n')
        
ViCol1TableWriter('./resources/employees.vicol1').insert_many(employees)

print('ViCol1 written successfully')

# write csv 

import csv
with open('resources/employees.csv', 'w') as f:
    w = csv.DictWriter(f, employees[0].keys())
    w.writeheader()
    w.writerows(employees)
    
    
print('CSV written successfully')

In [25]:
from csv_table import CsvTable

def profile_csv():
    (CsvTable('./resources/employees.csv')
        .where(lambda x: x['department'] == 'BigData')
        .select('salary')
        .collect_silent())



In [26]:
from vicol1_table import ViCol1TableReader

def profile_vicol1():
    (ViCol1TableReader('resources/employees.vicol1')
     .where('department', lambda x: x == 'BigData')
     .select(['salary'])
     .execute_silent())

# Time

In [30]:
%%timeit
profile_vicol1()

In [31]:
%%timeit
profile_csv()

# Memory

In [32]:
%load_ext memory_profiler

In [34]:
%memit profile_csv()

In [35]:
%memit profile_vicol1()